# Home Credit Default Risk

## Objective: 

**When there is less or no credit history available, use the alternative data like the telecommunication and transactional data to predict the clients' repayment capabilities.**

### About Data

#### Let's look at the data 

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline

import seaborn as sns

In [3]:
print "numpy:", np.__version__
print "pandas:", pd.__version__

numpy: 1.12.1
pandas: 0.20.3


#### Process all files

In [6]:
import os

In [39]:
INPUT_PATH = "./input/"

file_list = os.listdir(INPUT_PATH)

print file_list[0]
print os.listdir(INPUT_PATH)

application_train.csv
['application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv']


In [35]:
# read only 1000 records to easy visualizations..

app_train_df = pd.read_csv(INPUT_PATH+file_list[0]).sort_values(['SK_ID_CURR']).reset_index().loc[:1000, :].drop(['index'], axis=1)


In [52]:
prev_app_df = pd.read_csv(INPUT_PATH+file_list[6]).sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index().loc[:1000, :].drop(['index'], axis=1)

In [56]:
pos_cash_bal_df = pd.read_csv(INPUT_PATH+file_list[5]).sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index().loc[:1000, :].drop(['index'], axis=1)
credit_car_bal_df = pd.read_csv(INPUT_PATH+file_list[3]).sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index().loc[:1000, :].drop(['index'], axis=1)
install_payments_df = pd.read_csv(INPUT_PATH+file_list[4]).sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index().loc[:1000, :].drop(['index'], axis=1)

In [57]:
bureau_bal_df = pd.read_csv(INPUT_PATH+file_list[2]).sort_values(['SK_ID_BUREAU']).reset_index().loc[:1000, :].drop(['index'], axis=1)

In [58]:
bureau_df = pd.read_csv(INPUT_PATH+file_list[1]).sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index().loc[:1000, :].drop(['index'], axis=1)

In [53]:
bureau_df.shape

(1001, 17)

In [54]:
prev_app_df.shape

(1001, 37)

In [55]:
bureau_df.columns

Index([u'SK_ID_CURR', u'SK_ID_BUREAU', u'CREDIT_ACTIVE', u'CREDIT_CURRENCY',
       u'DAYS_CREDIT', u'CREDIT_DAY_OVERDUE', u'DAYS_CREDIT_ENDDATE',
       u'DAYS_ENDDATE_FACT', u'AMT_CREDIT_MAX_OVERDUE', u'CNT_CREDIT_PROLONG',
       u'AMT_CREDIT_SUM', u'AMT_CREDIT_SUM_DEBT', u'AMT_CREDIT_SUM_LIMIT',
       u'AMT_CREDIT_SUM_OVERDUE', u'CREDIT_TYPE', u'DAYS_CREDIT_UPDATE',
       u'AMT_ANNUITY'],
      dtype='object')

In [51]:
prev_app_df.columns

Index([u'SK_ID_PREV', u'SK_ID_CURR', u'NAME_CONTRACT_TYPE', u'AMT_ANNUITY',
       u'AMT_APPLICATION', u'AMT_CREDIT', u'AMT_DOWN_PAYMENT',
       u'AMT_GOODS_PRICE', u'WEEKDAY_APPR_PROCESS_START',
       u'HOUR_APPR_PROCESS_START', u'FLAG_LAST_APPL_PER_CONTRACT',
       u'NFLAG_LAST_APPL_IN_DAY', u'RATE_DOWN_PAYMENT',
       u'RATE_INTEREST_PRIMARY', u'RATE_INTEREST_PRIVILEGED',
       u'NAME_CASH_LOAN_PURPOSE', u'NAME_CONTRACT_STATUS', u'DAYS_DECISION',
       u'NAME_PAYMENT_TYPE', u'CODE_REJECT_REASON', u'NAME_TYPE_SUITE',
       u'NAME_CLIENT_TYPE', u'NAME_GOODS_CATEGORY', u'NAME_PORTFOLIO',
       u'NAME_PRODUCT_TYPE', u'CHANNEL_TYPE', u'SELLERPLACE_AREA',
       u'NAME_SELLER_INDUSTRY', u'CNT_PAYMENT', u'NAME_YIELD_GROUP',
       u'PRODUCT_COMBINATION', u'DAYS_FIRST_DRAWING', u'DAYS_FIRST_DUE',
       u'DAYS_LAST_DUE_1ST_VERSION', u'DAYS_LAST_DUE', u'DAYS_TERMINATION',
       u'NFLAG_INSURED_ON_APPROVAL'],
      dtype='object')

In [47]:
bureau_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,100001,5896633,Closed,currency 1,-1572,0,-1329.0,-1328.0,NaN,0,85500.0,0.0,0.0,0.0,Consumer credit,-155,0.0
1,100001,5896631,Closed,currency 1,-909,0,-179.0,-877.0,NaN,0,279720.0,0.0,0.0,0.0,Consumer credit,-155,0.0
2,100001,5896636,Active,currency 1,-320,0,411.0,NaN,NaN,0,168345.0,110281.5,NaN,0.0,Consumer credit,-10,9364.5
3,100001,5896635,Active,currency 1,-49,0,1778.0,NaN,NaN,0,378000.0,373239.0,0.0,0.0,Consumer credit,-16,10822.5
4,100001,5896632,Closed,currency 1,-879,0,-514.0,-544.0,NaN,0,91620.0,0.0,0.0,0.0,Consumer credit,-155,0.0


In [38]:
print len(app_train_df.columns)
print app_train_df.shape

122
(1001, 122)


In [27]:
#?pd.read_csv
?pd.drop()

Object `pd.drop` not found.


In [37]:
app_train_df.columns

Index([u'SK_ID_CURR', u'TARGET', u'NAME_CONTRACT_TYPE', u'CODE_GENDER',
       u'FLAG_OWN_CAR', u'FLAG_OWN_REALTY', u'CNT_CHILDREN',
       u'AMT_INCOME_TOTAL', u'AMT_CREDIT', u'AMT_ANNUITY',
       ...
       u'FLAG_DOCUMENT_18', u'FLAG_DOCUMENT_19', u'FLAG_DOCUMENT_20',
       u'FLAG_DOCUMENT_21', u'AMT_REQ_CREDIT_BUREAU_HOUR',
       u'AMT_REQ_CREDIT_BUREAU_DAY', u'AMT_REQ_CREDIT_BUREAU_WEEK',
       u'AMT_REQ_CREDIT_BUREAU_MON', u'AMT_REQ_CREDIT_BUREAU_QRT',
       u'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)

In [34]:
#app_train_df.drop(['index'], axis=1)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.000,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.000,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.000,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.000,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.000,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
5,100008,0,Cash loans,M,N,Y,0,99000.000,490495.5,27517.5,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
6,100009,0,Cash loans,F,Y,Y,1,171000.000,1560726.0,41301.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,2.0
7,100010,0,Cash loans,M,Y,Y,0,360000.000,1530000.0,42075.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
8,100011,0,Cash loans,F,N,Y,0,112500.000,1019610.0,33826.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
9,100012,0,Revolving loans,M,N,Y,0,135000.000,405000.0,20250.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
app_train_df.head()

,index,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


### Data types

In [112]:
print len(app_train_df.dtypes)
all_col_list = app_train_df.columns.tolist()
int_list = app_train_df.dtypes[app_train_df.dtypes == 'int64'].index.tolist()
float_list = app_train_df.dtypes[app_train_df.dtypes == 'float64'].index.tolist()
cat_list = app_train_df.dtypes[app_train_df.dtypes == 'object'].index.tolist()
print "int type:", len(int_list)
print "float type:", len(float_list)
print "categorical type:", len(cat_list)

122
int type: 41
float type: 65
categorical type: 16


In [121]:
# remove salary
#int_list.remove('TARGET')
int_list.remove('SK_ID_CURR')

In [114]:
#app_train_df.dtypes[app_train_df.dtypes == 'int64']

### Get correlation info

In [123]:
import math
from __future__ import division
from scipy.stats import pearsonr

In [59]:
col_list = app_train_df.columns
print col_list

Index([u'SK_ID_CURR', u'TARGET', u'NAME_CONTRACT_TYPE', u'CODE_GENDER',
       u'FLAG_OWN_CAR', u'FLAG_OWN_REALTY', u'CNT_CHILDREN',
       u'AMT_INCOME_TOTAL', u'AMT_CREDIT', u'AMT_ANNUITY',
       ...
       u'FLAG_DOCUMENT_18', u'FLAG_DOCUMENT_19', u'FLAG_DOCUMENT_20',
       u'FLAG_DOCUMENT_21', u'AMT_REQ_CREDIT_BUREAU_HOUR',
       u'AMT_REQ_CREDIT_BUREAU_DAY', u'AMT_REQ_CREDIT_BUREAU_WEEK',
       u'AMT_REQ_CREDIT_BUREAU_MON', u'AMT_REQ_CREDIT_BUREAU_QRT',
       u'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)


In [124]:
target = 'TARGET'

In [127]:
#get the correlation info
correlations = {} # dictionary
for f in int_list:
    data_temp = app_train_df[[f,target]]
    x1 = data_temp[f].values
    x2 = data_temp[target].values
    key = f + ' vs ' + target
    correlations[key] = pearsonr(x1,x2)[0]

In [128]:
correlations

{'CNT_CHILDREN vs TARGET': 0.025307164799204227,
 'DAYS_BIRTH vs TARGET': 0.08290688115231104,
 'DAYS_EMPLOYED vs TARGET': -0.010217880478518177,
 'DAYS_ID_PUBLISH vs TARGET': 0.055585456015736827,
 'FLAG_CONT_MOBILE vs TARGET': 0.015033826580673058,
 'FLAG_DOCUMENT_10 vs TARGET': nan,
 'FLAG_DOCUMENT_11 vs TARGET': -0.015033826580673053,
 'FLAG_DOCUMENT_12 vs TARGET': nan,
 'FLAG_DOCUMENT_13 vs TARGET': -0.012268922785474687,
 'FLAG_DOCUMENT_14 vs TARGET': -0.01226892278547469,
 'FLAG_DOCUMENT_15 vs TARGET': -0.0086710996952412012,
 'FLAG_DOCUMENT_16 vs TARGET': 0.011844202062334903,
 'FLAG_DOCUMENT_17 vs TARGET': nan,
 'FLAG_DOCUMENT_18 vs TARGET': 0.091700459338195289,
 'FLAG_DOCUMENT_19 vs TARGET': nan,
 'FLAG_DOCUMENT_2 vs TARGET': nan,
 'FLAG_DOCUMENT_20 vs TARGET': nan,
 'FLAG_DOCUMENT_21 vs TARGET': nan,
 'FLAG_DOCUMENT_3 vs TARGET': 0.020757005963141922,
 'FLAG_DOCUMENT_4 vs TARGET': nan,
 'FLAG_DOCUMENT_5 vs TARGET': -0.036041353404181221,
 'FLAG_DOCUMENT_6 vs TARGET': 0.0060

In [132]:
## go over float_list
for f in float_list:
    data_temp = app_train_df[[f,target]]
    x1 = data_temp[f].values
    x2 = data_temp[target].values
    key = f + ' vs ' + target
    correlations[key] = pearsonr(x1,x2)[0]

In [133]:
data_correlations = pd.DataFrame(correlations, index=['Value']).T
data_correlations.loc[data_correlations['Value'].abs().sort_values(ascending=False).index]

,Value
FLAG_DOCUMENT_18 vs TARGET,0.091700
DAYS_BIRTH vs TARGET,0.082907
DAYS_LAST_PHONE_CHANGE vs TARGET,0.074258
DAYS_ID_PUBLISH vs TARGET,0.055585
REG_REGION_NOT_LIVE_REGION vs TARGET,0.051329
REG_CITY_NOT_WORK_CITY vs TARGET,0.049930
FLAG_WORK_PHONE vs TARGET,0.049613
REGION_RATING_CLIENT vs TARGET,0.048279
REGION_RATING_CLIENT_W_CITY vs TARGET,0.047879
HOUR_APPR_PROCESS_START vs TARGET,-0.047849


In [134]:
print len(int_list + float_list)

104


In [135]:
print app_train_df.shape

(1001, 122)
